# 최신 코드 (25.04.30 기준)

In [1]:
%%capture
# @title install
# ======================= #
#   📦 필수 라이브러리 설치  #
# ======================= #
%pip install -q pyngrok huggingface_hub peft transformers
%pip install -q -U "vllm[triton]" accelerate
%pip install -q pyzmq==25.1.2

In [2]:
# @title Setting
# ======================= #
#   🔧 설정값 세팅 및 로그인  #
# ======================= #
import os
import time
import subprocess
import logging
from google.colab import userdata
from huggingface_hub import login
from pyngrok import ngrok
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Secret 가져오기
hf_token = userdata.get("HF_TOKEN")
ngrok_token = userdata.get("naver_ngrok_authtoken")

# model path
base_model_repo = "unsloth/Llama-3.2-3B-Instruct"
lora_repo = "rnl000/llama-lora"
# model_name = 'colab-llama'    # vLLM 이슈로 일단 스킵

# more setting
log_path = "vllm.log"
vllm_port = 8000
STATIC_NGROK_DOMAIN = "serval-knowing-hamster.ngrok-free.app"  # 고정 도메인 사용 시 (선택)

# 로그인
login(token=hf_token)

In [3]:
# @title Load Model
# ======================= #
#   🚀 모델 로드 및 병합    #
# ======================= #
print("🚀 모델 로드 및 병합 중...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_repo,
    torch_dtype=torch.float16,
    device_map="cpu"
)

tokenizer = AutoTokenizer.from_pretrained(base_model_repo, use_fast=True)

model = PeftModel.from_pretrained(
    base_model,
    lora_repo,
    torch_dtype=torch.float16,
    device_map="cpu"
)

merged_model_path = "./merged_model"
if os.path.exists(merged_model_path):
    import shutil
    shutil.rmtree(merged_model_path)

merged_model = model.merge_and_unload()
merged_model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

print("✅ 병합 완료 및 저장됨.")

🚀 모델 로드 및 병합 중...


config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/800 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

✅ 병합 완료 및 저장됨.


In [4]:
# @title Fin.
import subprocess
from pyngrok import ngrok

# ngrok 프로세스 종료
try:
    ngrok.kill()
    print("✅ ngrok 프로세스 종료 완료")
except Exception as e:
    print(f"⚠️ ngrok 종료 중 오류 (무시 가능): {e}")


✅ ngrok 프로세스 종료 완료


In [5]:
# @title vLLM Server Start
# ======================= #
#   🚀 vLLM 서버 실행      #
# ======================= #
print("🚀 vLLM 서버 시작 중...")
if os.path.exists(log_path):
    os.remove(log_path)

subprocess.Popen(
    f"python3 -m vllm.entrypoints.openai.api_server "
    f"--model {merged_model_path} --dtype float16 "
    f"--port {vllm_port} --gpu-memory-utilization 0.6 "
    f"--max-model-len 4096 > {log_path} 2>&1 &",
    shell=True
)

# vLLM 준비 대기 (로그 기반)
print("⏳ vLLM 서버 준비 대기 중...")
ready = False
for i in range(150):
    if os.path.exists(log_path):
        with open(log_path) as f:
            logs = f.read()
            if "Application startup complete" in logs:
                print("✅ vLLM 서버가 실행 중입니다!")
                ready = True
                break
    time.sleep(1)

if not ready:
    print("⚠️ vLLM 서버가 아직 준비되지 않았습니다. 로그 확인 필요")
    !tail -n 20 vllm.log



# ======================= #
#   🌎 ngrok 연결 시작     #
# ======================= #
print("🔑 ngrok 연결 중...")
ngrok.set_auth_token(ngrok_token)
try:
    public_url = ngrok.connect(
        addr=vllm_port,
        proto="http",
        domain=STATIC_NGROK_DOMAIN  # 고정 도메인 (선택)
    ).public_url
except:
    public_url = ngrok.connect(vllm_port, "http").public_url

print(f"✅ ngrok 연결 완료: {public_url}")

# ======================= #
#   🔍 상태 확인 및 출력   #
# ======================= #
time.sleep(5)
!curl -s http://127.0.0.1:8000/v1/models

print("\n🔗 외부 접속 주소:", public_url)
!ps -ef | grep -E "vllm|ngrok" | grep -v grep

🚀 vLLM 서버 시작 중...
⏳ vLLM 서버 준비 대기 중...
✅ vLLM 서버가 실행 중입니다!
🔑 ngrok 연결 중...
✅ ngrok 연결 완료: https://serval-knowing-hamster.ngrok-free.app
{"object":"list","data":[{"id":"./merged_model","object":"model","created":1746104883,"owned_by":"vllm","root":"./merged_model","parent":null,"max_model_len":4096,"permission":[{"id":"modelperm-74d7256a76b44a2d89f78a71752be786","object":"model_permission","created":1746104883,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}
🔗 외부 접속 주소: https://serval-knowing-hamster.ngrok-free.app
root        4472       1 11 13:05 ?        00:00:17 python3 -m vllm.entrypoints.openai.api_server --model ./merged_model --dtype float16 --port 8000 --gpu-memory-utilization 0.6 --max-model-len 4096
root        5411    3314  2 13:07 ?        00:00:00 /root/.config/ngrok/ngrok start --none --log=stdout


In [6]:
!nvidia-smi

Thu May  1 13:08:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             50W /  400W |   24820MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# 이전 코드 (25.04.30 기준)

In [7]:
# %%capture
# # ======================= #
# #   📦 필수 라이브러리 설치  #
# # ======================= #
# %pip install pyngrok -q
# %pip install huggingface_hub -q
# %pip install peft transformers -q

In [8]:
# # ======================= #
# #   🔧 설정값 세팅 및 로그인  #
# # ======================= #
# import os
# from google.colab import userdata
# from huggingface_hub import login

# # Hugging Face 및 ngrok 토큰 (Colab Secret에서 가져오기)
# hf_token = userdata.get("HF_TOKEN")        # Hugging Face Token
# ngrok_token = userdata.get("naver_ngrok_authtoken")  # ngrok Token

# if hf_token == None:
#   print('x')
# if ngrok_token == None:
#   print('x')

# # Hugging Face 모델 경로
# base_model_repo = "unsloth/Llama-3.2-3B-Instruct"
# lora_repo = "rnl000/llama-lora"  # 본인이 업로드한 LoRA repo

# # vLLM 서버 설정
# vllm_port = 8000

# # 로그인
# print("🔑 Hugging Face 로그인 중...")
# login(token=hf_token)

In [9]:
# # ======================= #
# #   🚀 모델 로드 (Base + LoRA)  #
# # ======================= #
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel
# import torch

# # Base 모델 로딩
# print("🚀 Base 모델 로딩 중...")
# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_repo,
#     torch_dtype=torch.float16,
#     device_map="cpu"    # 초기 cpu 적제
# )


# # Tokenizer 로딩
# print("🚀 Tokenizer 로딩 중...")
# tokenizer = AutoTokenizer.from_pretrained(
#     base_model_repo,
#     use_fast=True
# )

# # LoRA Adapter 로딩
# print("🚀 LoRA adapter 로딩 중...")
# model = PeftModel.from_pretrained(
#     base_model,
#     lora_repo,
#     torch_dtype=torch.float16,
#     device_map="cpu"   # LoRA도 cpu
# )


# print("✅ Base + LoRA 모델 로딩 완료!")


In [10]:
# # ======================= #
# #   🔀 Base + LoRA 모델 병합 및 저장  #
# # ======================= #
# import os

# merged_model_path = "./merged_model"

# # 이미 폴더가 있다면 삭제 후 생성 (오류 방지용)
# if os.path.exists(merged_model_path):
#     import shutil
#     shutil.rmtree(merged_model_path)

# # 모델 병합
# print("🔀 Base + LoRA 모델 병합 중...")
# merged_model = model.merge_and_unload()

# # 저장
# merged_model.save_pretrained(merged_model_path)
# tokenizer.save_pretrained(merged_model_path)

# print(f"✅ 병합된 모델 저장 완료: {merged_model_path}")


In [11]:
# %%capture
# ''' vLLM 오류 해결을 위함 '''
# %pip install -U "vllm[triton]" accelerate
# %pip install pyzmq==25.1.2

In [12]:
# # ======================= #
# #   🚀 vLLM 서버 실행하기  #
# # ======================= #

# import time

# print("🚀 vLLM 서버를 백그라운드에서 시작합니다...")
# !python3 -m vllm.entrypoints.openai.api_server \
#   --model ./merged_model \
#   --dtype float16 \
#   --port {vllm_port} \
#   --gpu-memory-utilization 0.6 \
#   --max-model-len 4096 > vllm.log 2>&1 &

# print("⏳ vLLM 서버 시작 대기 중 (약 90초)...")
# time.sleep(90)



# # ======================= #
# #   🌎 ngrok 연결 설정 및 시작  #
# # ======================= #
# from pyngrok import ngrok

# print("🔑 ngrok Authtoken 설정 중...")
# try:
#     ngrok.set_auth_token(ngrok_token)
#     print("✅ ngrok Authtoken 설정 완료.")
# except Exception as e:
#     print(f"⚠️ ngrok Authtoken 설정 실패 (무시 가능): {e}")

# print(f"🚇 ngrok 터널을 포트 {vllm_port}에 연결합니다...")
# try:
#     tunnel = ngrok.connect(vllm_port, "http")
#     public_url = tunnel.public_url
#     print(f"✅ ngrok 터널 생성 완료: {public_url}")
# except Exception as e:
#     print(f"❌ ngrok 터널 생성 실패: {e}")
#     public_url = None


# # HTTP 헬스체크: 5초 정도 기다렸다가 호출
# !sleep 5
# !curl -v http://127.0.0.1:8000/v1/models

# # ngrok 프로세스 및 터널 정보 확인 (선택 사항)
# print("\n🔍 실행 중인 ngrok 프로세스 확인:")
# !ps -ef | grep ngrok

# print("\n📊 활성 ngrok 터널 정보:")
# # get_tunnels()는 활성화된 터널 객체 리스트를 반환합니다.
# active_tunnels = ngrok.get_tunnels()
# if active_tunnels:
#     for t in active_tunnels:
#         print(t)
# else:
#     print("활성 ngrok 터널이 없습니다.")
# print('\n\n\n==================')


# # 최종 출력
# if public_url:
#     print(f"\n✅ vLLM 서버가 {public_url} 에서 실행 중입니다.")
#     print("   (Colab 셀 실행이 중지되면 터널도 함께 종료됩니다.)")
#     input("\n🔵 서버를 종료하려면 엔터를 누르세요...")
# else:
#     print("\n❌ 터널 URL을 가져오지 못했습니다. 로그를 확인하세요.")
#     !tail -n 20 vllm.log


In [13]:
# # 확인용
# !python3 -m vllm.entrypoints.openai.api_server \
#   --model ./merged_model \
#   --dtype float16 \
#   --port {vllm_port} \
#   --gpu-memory-utilization 0.6 \
#   --max-model-len 4096

# print("⏳ vLLM 서버 시작 대기 중 (약 20초)...")